<a href="https://colab.research.google.com/github/hansinihirunika/PneumoniaDetection/blob/main/DetectPneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [ ]:
!nvidia-smi

Wed Feb  8 06:50:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
#re-size the images
IMAGE_SIZE = [224, 224]
#path to the datasets
traindataset_path = "/content/drive/MyDrive/Programs/Datasets/Train"
testdataset_path = "/content/drive/MyDrive/Programs/Datasets/Test"

In [ ]:
#ResNet library 
#Transfer learning technique
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
#existing weights as false
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
#the number of output classes in Train
folders = glob("/content/drive/MyDrive/Programs/Datasets/Train/*")

In [ ]:
folders

['/content/drive/MyDrive/Programs/Datasets/Train/bacteria',
 '/content/drive/MyDrive/Programs/Datasets/Train/virus',
 '/content/drive/MyDrive/Programs/Datasets/Train/normal']

In [ ]:
#the number of output classes in Test
folders = glob("/content/drive/MyDrive/Programs/Datasets/Test/*")

In [ ]:
folders

['/content/drive/MyDrive/Programs/Datasets/Test/virus',
 '/content/drive/MyDrive/Programs/Datasets/Test/normal',
 '/content/drive/MyDrive/Programs/Datasets/Test/bacteria']

In [ ]:
#flatten the output / adding the last layer
y = Flatten()(resnet.output)

In [ ]:
#how many output nodes that want 
#softmax use for multiple categories
prediction = Dense(len(folders), activation='softmax')(y)

# creating a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
#the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
#optimization method
model.compile(
    loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = "accuracy"
)

In [ ]:
# Image Data Generator - imports images from dataset
#data argumentation techniques
from tensorflow.keras.preprocessing.image import ImageDataGenerator
trainDatagenerator = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2
)

# only scaling process for test
testDatagenerator = ImageDataGenerator(rescale = 1./255)

In [ ]:
# reading the train dataset
trainingDataset = trainDatagenerator.flow_from_directory(
    "/content/drive/MyDrive/Programs/Datasets/Train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 5236 images belonging to 3 classes.


In [ ]:
# reading the test dataset
testingDataset = testDatagenerator.flow_from_directory(
    "/content/drive/MyDrive/Programs/Datasets/Test",
    target_size = (224, 224),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 624 images belonging to 3 classes.


In [ ]:
#fitting model
r = model.fit_generator(
    trainingDataset,
    validation_data=testingDataset,
    epochs=50,
    steps_per_epoch=len(trainingDataset),
    validation_steps=len(testingDataset)
)

<ipython-input-19-cbda0b521b59>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
328/328 [==============================] - 3716s 11s/step - loss: 1.5073 - accuracy: 0.5088 - val_loss: 0.7255 - val_accuracy: 0.6987
Epoch 2/50
328/328 [==============================] - 117s 357ms/step - loss: 1.1606 - accuracy: 0.5900 - val_loss: 0.6300 - val_accuracy: 0.7708
Epoch 3/50
328/328 [==============================] - 117s 358ms/step - loss: 0.9618 - accuracy: 0.6369 - val_loss: 0.5624 - val_accuracy: 0.7981
Epoch 4/50
328/328 [==============================] - 117s 355ms/step - loss: 0.9393 - accuracy: 0.6385 - val_loss: 1.2897 - val_accuracy: 0.6571
Epoch 5/50
328/328 [==============================] - 117s 358ms/step - loss: 1.0647 - accuracy: 0.6453 - val_loss: 0.8191 - val_accuracy: 0.6875
Epoch 6/50
328/328 [==============================] - 119s 362ms/step - loss: 1.0405 - accuracy: 0.6417 - val_loss: 0.8251 - val_accuracy: 0.7468
Epoch 7/50
328/328 [==============================] - 117s 356ms/step - loss: 1.0961 - accuracy: 0.6469 - val_loss: 1.8285 - 